In [1]:
import os

In [2]:
%cd /home/jupyter-23523024/Eksperimen/amr-tst-indo

/home/jupyter-23523024/Eksperimen/amr-tst-indo


/opt/tljh/user/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [4]:
!export CUDA_VISIBLE_DEVICES=2

In [5]:
PROJECT_ROOT = "/home/jupyter-23523024/Eksperimen/amr-tst-indo"

In [6]:
!git pull

Already up to date.


In [7]:
# !pip install -r requirements.txt

# Initialization

## AMR Parsing

In [8]:
from huggingface_hub import snapshot_download

# Gold v2
amr_parsing_model_name = "mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan"
snapshot_download(
    repo_id=f"abdiharyadi/{amr_parsing_model_name}",
    local_dir=f"{PROJECT_ROOT}/AMRBART-id/models/{amr_parsing_model_name}",
    ignore_patterns=[
        "*log*",
        "*checkpoint*",
    ]
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

'/home/jupyter-23523024/Eksperimen/amr-tst-indo/AMRBART-id/models/mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan'

In [9]:
from text_to_amr import TextToAMR
t2a = TextToAMR(model_name=amr_parsing_model_name)  # Gold v2

/home/jupyter-23523024/.local/lib/python3.10/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
2025-03-16 07:31:57.716954: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-16 07:31:57.734445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742085117.755604 2312904 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742085117.766793 2312904 cuda_

In [10]:
import penman
graphs = t2a(["Halo, dunia!"])
print(penman.encode(graphs[0]))

Generating test split: 0 examples [00:00, ? examples/s]

Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

# ::id 0
# ::annotator mbart-en-id-smaller-indo-amr-parsing-translated-nafkhan
# ::snt Halo, dunia!
(z0 / dunia
    :mode ekspresif
    :mod (z1 / Halo
             :mode ekspresif))


## AMR-To-Text (AMR Generation)

### Using Taufiq's method (current)

In [11]:
from amr_to_text import AMRToTextWithTaufiqMethod
amr_gen_model_name = "taufiq-indo-amr-generation-inf-gold-v4.cl-uncased"
model_path = f"./{amr_gen_model_name}"

In [12]:
from huggingface_hub import snapshot_download
snapshot_download(
    repo_id=f"atikaistiqomah/{amr_gen_model_name}",
    local_dir=model_path,
    allow_patterns=[
        "*checkpoint-3*"
    ]
)

Fetching 88 files:   0%|          | 0/88 [00:00<?, ?it/s]

'/home/jupyter-23523024/Eksperimen/amr-tst-indo/taufiq-indo-amr-generation-inf-gold-v4.cl-uncased'

In [13]:
a2t = AMRToTextWithTaufiqMethod(
    model_path=os.path.join(model_path, "checkpoint-3"),
    lowercase=True,
)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


Running on the GPU


## AMR-TST

In [14]:
from amr_tst_new import AMRTST
amr_tst = AMRTST(t2a=t2a, a2t=a2t)
amr_tst

# Normal inference
1. Edit `sentences`.
2. Edit `source_styles`. `LABEL_0` for negative, and `LABEL_1` for positive.
3. Execute.

In [15]:
sentences = [
    "tolong admin untuk melihat pesan dari saya .",
    "saya tidak suka belajar.",
    "kenapa kamu tidak pergi ke sekolah ?"
]

results, infos = amr_tst(sentences)
results

Generating test split: 0 examples [00:00, ? examples/s]

Running tokenizer on prediction dataset:   0%|          | 0/3 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3 [00:00<?, ? examples/s]

100%|██████████| 3/3 [00:00<00:00,  4.43it/s]


['tolong admin lihat pesan saya',
 'aku ga suka belajar',
 'kenapa kamu gabisa ke sekolah ?']

In [16]:
infos.to_list()[0]

{'source_text': 'tolong admin untuk melihat pesan dari saya .',
 'source_amr': '(z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / lihat-01 :ARG0 (z3 / orang :ARG0-of (z4 / punya-peran-org-91 :ARG2 (z5 / admin))) :ARG1 (z6 / pesan :source z1)))',
 'target_amr': '(z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / lihat-01 :ARG0 (z3 / orang :ARG0-of (z4 / punya-peran-org-91 :ARG2 (z5 / admin))) :ARG1 (z6 / pesan :source z1)))',
 'target_text': 'tolong admin lihat pesan saya'}

In [17]:
for single_info in infos.to_list():
    x_src = single_info["source_text"]
    g_src_str = single_info["source_amr"]
    g_tgt_str = single_info["target_amr"]
    x_tgt = single_info["target_text"]

    print("--- [input] Teks sumber:", x_src)
    print("--- Graf sumber:", g_src_str)
    print("--- Graf target:", g_tgt_str)
    print("--- [output] Teks target:", x_tgt)
    print("=======")

--- [input] Teks sumber: tolong admin untuk melihat pesan dari saya .
--- Graf sumber: (z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / lihat-01 :ARG0 (z3 / orang :ARG0-of (z4 / punya-peran-org-91 :ARG2 (z5 / admin))) :ARG1 (z6 / pesan :source z1)))
--- Graf target: (z0 / tolong-01 :ARG0 (z1 / aku) :ARG1 (z2 / lihat-01 :ARG0 (z3 / orang :ARG0-of (z4 / punya-peran-org-91 :ARG2 (z5 / admin))) :ARG1 (z6 / pesan :source z1)))
--- [output] Teks target: tolong admin lihat pesan saya
--- [input] Teks sumber: saya tidak suka belajar.
--- Graf sumber: (z0 / suka-01 :polarity - :ARG0 (z1 / aku) :ARG1 (z2 / belajar-01))
--- Graf target: (z0 / suka-01 :polarity - :ARG0 (z1 / aku) :ARG1 (z2 / belajar-01))
--- [output] Teks target: aku ga suka belajar
--- [input] Teks sumber: kenapa kamu tidak pergi ke sekolah ?
--- Graf sumber: (z0 / sebab-01 :ARG0 (z1 / amr-tidak-diketahui) :ARG1 (z2 / pergi-01 :polarity - :ARG0 (z3 / kamu) :ARG4 (z4 / sekolah)))
--- Graf target: (z0 / sebab-01 :ARG0 (z1 / amr-tidak-d

# From file

## Without precomputed results, per data.

In [18]:
filename0 = "/home/jupyter-23523024/Dataset/amr-tst-paralel/hasil_paralel_cl_formal-to-informal_test.txt"

In [19]:
sentences = []
targets = []

with open(filename0, encoding="utf-8") as fp:
    for line in fp:
        line = line.strip()
        first_sentence, second_sentence = line.split("\t")  # Formal -> Informal
        sentences.append(first_sentence)
        targets.append(second_sentence)

len(sentences), len(targets)

(363, 363)

In [20]:
from tqdm import tqdm
all_results = []
all_infos = []

In [21]:
for i, (x, y) in enumerate(tqdm(zip(sentences, targets), total=len(sentences))):
    if i < len(all_infos):
        continue

    results, infos = amr_tst([x])  # Tanpa informasi style
    all_results.extend(results)

    info = infos.to_list()[0]
    info["human_target_text"] = y
    all_infos.append(info)

len(all_infos)

  0%|          | 0/363 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Running tokenizer on prediction dataset:   0%|          | 0/1 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



100%|██████████| 363/363 [16:07<00:00,  2.67s/it]


363

In [26]:
import json
with open("/home/jupyter-23523024/Eksperimen/amr_par-gen_formal-to-informal_result-eks 3-sentence-cl-join-gen-inf-test.json", mode="w") as fp:
    json.dump(all_infos, fp, indent=2)

In [27]:
from evaluation import compute_metrics_generation
preds = [x["target_text"] for x in all_infos]
references = [x["human_target_text"] for x in all_infos]

In [28]:
# For BLEU-r
compute_metrics_generation(preds, references)

{'score': 9.631670083749844,
 'counts': [2048, 664, 259, 108],
 'totals': [4863, 4500, 4137, 3774],
 'precisions': [42.11392144766605,
  14.755555555555556,
  6.260575296108291,
  2.861685214626391],
 'bp': 0.9376640389278643,
 'sys_len': 4863,
 'ref_len': 5176}

In [29]:
# For BLEU-s
compute_metrics_generation(preds, [x["source_text"] for x in all_infos])

{'score': 13.940733143743635,
 'counts': [2719, 1040, 449, 201],
 'totals': [4863, 4500, 4137, 3774],
 'precisions': [55.911988484474605,
  23.11111111111111,
  10.853275320280396,
  5.325914149443562],
 'bp': 0.8432676260532742,
 'sys_len': 4863,
 'ref_len': 5692}